# T3 Food Trucks - Data Exploration

This notebook explores the historical transaction data for Tasty Truck Treats (T3) to identify trends, patterns, and insights that will inform business recommendations.

## 1. Import Libraries

In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)
pd.set_option('display.precision', 2)

## 2. Load Clean Data

In [ ]:
df = pd.read_csv('data/clean/combined_data.csv')
df['at'] = pd.to_datetime(df['at'])

trucks = pd.read_csv('data/clean/trucks_clean.csv')
payment_methods = pd.read_csv('data/clean/payment_methods_clean.csv')
transactions = pd.read_csv('data/clean/transactions_clean.csv')
transactions['at'] = pd.to_datetime(transactions['at'])

print(f"Total records: {len(df):,}")
print(f"Date range: {df['at'].min()} to {df['at'].max()}")

Total records: 5,472
Date range: 2026-01-04 09:03:00 to 2026-01-10 23:56:00


## 3. Data Overview

In [ ]:
print("Sample Data:")
display(df.head())

print("\nData Types:")
display(df.dtypes)

print("\nMissing Values:")
display(df.isnull().sum())

Sample Data:


,transaction_id,truck_id,payment_method_id,total,at,truck_name,truck_description,has_card_reader,fsa_rating,payment_method
0,1,1,2,700.0,2026-01-04 09:04:00,Burrito Madness,An authentic taste of Mexico.,True,4,Card
1,2,1,1,700.0,2026-01-04 09:22:00,Burrito Madness,An authentic taste of Mexico.,True,4,Cash
2,3,1,1,700.0,2026-01-04 09:26:00,Burrito Madness,An authentic taste of Mexico.,True,4,Cash
3,4,1,2,869.0,2026-01-04 09:41:00,Burrito Madness,An authentic taste of Mexico.,True,4,Card
4,5,1,1,700.0,2026-01-04 09:56:00,Burrito Madness,An authentic taste of Mexico.,True,4,Cash



Data Types:


transaction_id                int64
truck_id                      int64
payment_method_id             int64
total                       float64
at                   datetime64[ns]
truck_name                   object
truck_description            object
has_card_reader                bool
fsa_rating                    int64
payment_method               object
dtype: object


Missing Values:


transaction_id       0
truck_id             0
payment_method_id    0
total                0
at                   0
truck_name           0
truck_description    0
has_card_reader      0
fsa_rating           0
payment_method       0
dtype: int64

In [ ]:
print("Summary Statistics:")
display(df.describe())

Summary Statistics:


,transaction_id,truck_id,payment_method_id,total,at,fsa_rating
count,5472.00,5472.00,5472.00,5472.00,5472,5472.00
mean,2736.50,3.03,1.46,644.72,2026-01-07 08:48:09.057017600,3.82
min,1.00,1.00,1.00,99.00,2026-01-04 09:03:00,2.00
25%,1368.75,1.00,1.00,500.00,2026-01-05 12:35:00,3.00
50%,2736.50,3.00,1.00,700.00,2026-01-07 12:42:30,4.00
75%,4104.25,5.00,2.00,799.00,2026-01-08 23:41:00,4.00
max,5472.00,6.00,2.00,1299.00,2026-01-10 23:56:00,5.00
std,1579.77,1.69,0.50,228.40,NaN,0.98


## 4. Business Metrics Overview

In [ ]:
total_revenue = df['total'].sum()
total_transactions = len(df)
avg_transaction = df['total'].mean()
num_trucks = df['truck_id'].nunique()
num_days = (df['at'].max() - df['at'].min()).days

print("="*50)
print("KEY BUSINESS METRICS")
print("="*50)
print(f"Total Revenue: £{total_revenue:,.2f}")
print(f"Total Transactions: {total_transactions:,}")
print(f"Average Transaction Value: £{avg_transaction:.2f}")
print(f"Number of Active Trucks: {num_trucks}")
print(f"Period: {num_days} days")
print(f"Daily Average Revenue: £{total_revenue/num_days:,.2f}")
print("="*50)

KEY BUSINESS METRICS
Total Revenue: £3,527,899.00
Total Transactions: 5,472
Average Transaction Value: £644.72
Number of Active Trucks: 6
Period: 6 days
Daily Average Revenue: £587,983.17


## 5. Temporal Analysis - Sales Over Time

In [ ]:
daily_revenue = df.groupby(df['at'].dt.date)['total'].sum().reset_index()
daily_revenue.columns = ['date', 'revenue']

fig = px.line(daily_revenue, x='date', y='revenue',
              title='Daily Revenue Trend',
              labels={'revenue': 'Revenue (£)', 'date': 'Date'})
fig.update_layout(hovermode='x unified')
fig.show()

In [ ]:
df['hour'] = df['at'].dt.hour
hourly_sales = df.groupby('hour')['total'].agg(['sum', 'count', 'mean']).reset_index()
hourly_sales.columns = ['hour', 'total_revenue', 'transaction_count', 'avg_transaction']

fig = make_subplots(rows=1, cols=2,
                    subplot_titles=('Revenue by Hour', 'Transactions by Hour'))

fig.add_trace(
    go.Bar(x=hourly_sales['hour'], y=hourly_sales['total_revenue'], name='Revenue'),
    row=1, col=1
)

fig.add_trace(
    go.Bar(x=hourly_sales['hour'], y=hourly_sales['transaction_count'], name='Transactions'),
    row=1, col=2
)

fig.update_layout(title_text='Sales Patterns by Hour of Day', showlegend=False)
fig.show()

In [ ]:
df['day_of_week'] = df['at'].dt.day_name()
day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

daily_pattern = df.groupby('day_of_week')['total'].agg(['sum', 'count', 'mean']).reindex(day_order)
daily_pattern.columns = ['total_revenue', 'transaction_count', 'avg_transaction']
daily_pattern = daily_pattern.reset_index()

fig = px.bar(daily_pattern, x='day_of_week', y='total_revenue',
             title='Revenue by Day of Week',
             labels={'total_revenue': 'Total Revenue (£)', 'day_of_week': 'Day'})
fig.show()

## 6. Truck Performance Analysis

In [ ]:
truck_performance = df.groupby(['truck_id', 'truck_name']).agg({
    'total': ['sum', 'count', 'mean'],
    'transaction_id': 'count'
}).reset_index()

truck_performance.columns = ['truck_id', 'truck_name', 'total_revenue', 'transaction_count', 'avg_transaction', 'num_transactions']
truck_performance = truck_performance.sort_values('total_revenue', ascending=False)

print("Top Performing Trucks:")
display(truck_performance.head(10))

Top Performing Trucks:


,truck_id,truck_name,total_revenue,transaction_count,avg_transaction,num_transactions
0,1,Burrito Madness,1.10e+06,1416,775.25,1416
2,3,Cupcakes by Michelle,7.90e+05,1354,583.79,1354
1,2,Kings of Kebabs,7.17e+05,888,807.62,888
4,5,Yoghurt Heaven,5.00e+05,924,541.58,924
5,6,SuperSmoothie,3.31e+05,564,587.48,564
3,4,Hartmann's Jellied Eels,9.08e+04,326,278.45,326


In [ ]:
fig = px.bar(truck_performance.head(10), 
             x='truck_name', y='total_revenue',
             title='Top 10 Trucks by Revenue',
             labels={'total_revenue': 'Total Revenue (£)', 'truck_name': 'Truck Name'},
             color='total_revenue',
             color_continuous_scale='Blues')
fig.update_layout(xaxis_tickangle=-45)
fig.show()

In [ ]:
fig = px.box(df, x='truck_name', y='total',
             title='Transaction Value Distribution by Truck',
             labels={'total': 'Transaction Value (£)', 'truck_name': 'Truck'})
fig.update_layout(xaxis_tickangle=-45)
fig.show()

## 7. Payment Method Analysis

In [ ]:
payment_analysis = df.groupby('payment_method').agg({
    'total': ['sum', 'count', 'mean']
}).reset_index()

payment_analysis.columns = ['payment_method', 'total_revenue', 'transaction_count', 'avg_transaction']
payment_analysis['percentage'] = (payment_analysis['transaction_count'] / payment_analysis['transaction_count'].sum() * 100)

print("Payment Method Analysis:")
display(payment_analysis)

Payment Method Analysis:


,payment_method,total_revenue,transaction_count,avg_transaction,percentage
0,Card,1.63e+06,2511,649.83,45.89
1,Cash,1.90e+06,2961,640.38,54.11


In [ ]:
fig = px.pie(payment_analysis, values='transaction_count', names='payment_method',
             title='Transaction Count by Payment Method',
             hole=0.3)
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.show()

In [ ]:
fig = px.bar(payment_analysis, x='payment_method', y='total_revenue',
             title='Revenue by Payment Method',
             labels={'total_revenue': 'Total Revenue (£)', 'payment_method': 'Payment Method'},
             color='total_revenue',
             color_continuous_scale='Greens')
fig.show()

## 8. Card Reader Impact Analysis

In [ ]:
if 'has_card_reader' in df.columns:
    card_reader_impact = df.groupby('has_card_reader').agg({
        'total': ['sum', 'count', 'mean']
    }).reset_index()
    
    card_reader_impact.columns = ['has_card_reader', 'total_revenue', 'transaction_count', 'avg_transaction']
    
    print("Card Reader Impact:")
    display(card_reader_impact)
    
    # Visualization
    fig = px.bar(card_reader_impact, x='has_card_reader', y='avg_transaction',
                 title='Average Transaction Value: Card Reader vs No Card Reader',
                 labels={'avg_transaction': 'Average Transaction (£)', 'has_card_reader': 'Has Card Reader'})
    fig.show()

Card Reader Impact:


,has_card_reader,total_revenue,transaction_count,avg_transaction
0,False,3.31e+05,564,587.48
1,True,3.20e+06,4908,651.30


## 9. FSA Rating Analysis

In [ ]:
if 'fsa_rating' in df.columns:
    fsa_analysis = df.groupby('fsa_rating').agg({
        'total': ['sum', 'count', 'mean']
    }).reset_index()
    
    fsa_analysis.columns = ['fsa_rating', 'total_revenue', 'transaction_count', 'avg_transaction']
    
    print("FSA Rating Analysis:")
    display(fsa_analysis)
    
    # Visualization
    fig = px.scatter(fsa_analysis, x='fsa_rating', y='avg_transaction', size='transaction_count',
                     title='Transaction Value by FSA Rating',
                     labels={'avg_transaction': 'Average Transaction (£)', 'fsa_rating': 'FSA Rating'})
    fig.show()

FSA Rating Analysis:


,fsa_rating,total_revenue,transaction_count,avg_transaction
0,2,7.17e+05,888,807.62
1,3,3.31e+05,564,587.48
2,4,1.69e+06,2666,633.51
3,5,7.90e+05,1354,583.79


## 10. Revenue Distribution Analysis

In [ ]:
fig = px.histogram(df, x='total', nbins=50,
                   title='Distribution of Transaction Values',
                   labels={'total': 'Transaction Value (£)'})
fig.show()

# Statistics
print(f"Median Transaction: £{df['total'].median():.2f}")
print(f"25th Percentile: £{df['total'].quantile(0.25):.2f}")
print(f"75th Percentile: £{df['total'].quantile(0.75):.2f}")

Median Transaction: £700.00
25th Percentile: £500.00
75th Percentile: £799.00


## 11. Summary & Key Insights

In [93]:
print("="*70)
print("KEY INSIGHTS SUMMARY")
print("="*70)

print("\n1. REVENUE METRICS:")
print(f"   - Total Revenue: £{total_revenue:,.2f}")
print(f"   - Average Daily Revenue: £{total_revenue/num_days:,.2f}")
print(f"   - Average Transaction: £{avg_transaction:.2f}")

print("\n2. TOP PERFORMING TRUCK:")
top_truck = truck_performance.iloc[0]
print(f"   - Name: {top_truck['truck_name']}")
print(f"   - Revenue: £{top_truck['total_revenue']:,.2f}")
print(f"   - Transactions: {top_truck['transaction_count']:,}")

print("\n3. PAYMENT METHODS:")
for _, row in payment_analysis.iterrows():
    print(f"   - {row['payment_method']}: {row['percentage']:.1f}% of transactions")

print("\n4. PEAK HOURS:")
peak_hour = hourly_sales.loc[hourly_sales['total_revenue'].idxmax()]
print(f"   - Highest revenue hour: {int(peak_hour['hour'])}:00")
print(f"   - Peak hour revenue: £{peak_hour['total_revenue']:,.2f}")

print("\n5. BUSIEST DAY:")
busiest_day = daily_pattern.loc[daily_pattern['total_revenue'].idxmax()]
print(f"   - Day: {busiest_day['day_of_week']}")
print(f"   - Average revenue: £{busiest_day['total_revenue']:,.2f}")

print("\n" + "="*70)

KEY INSIGHTS SUMMARY

1. REVENUE METRICS:
   - Total Revenue: £3,527,899.00
   - Average Daily Revenue: £587,983.17
   - Average Transaction: £644.72

2. TOP PERFORMING TRUCK:
   - Name: Burrito Madness
   - Revenue: £1,097,748.00
   - Transactions: 1,416

3. PAYMENT METHODS:
   - Card: 45.9% of transactions
   - Cash: 54.1% of transactions

4. PEAK HOURS:
   - Highest revenue hour: 14:00
   - Peak hour revenue: £726,066.00

5. BUSIEST DAY:
   - Day: Sunday
   - Average revenue: £686,494.00

